In [4]:
!pip install catboost

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import time



from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV

In [6]:
df_train = pd.read_csv('/kaggle/input/blackfriday-scaled/BF_train_scaled.csv',index_col='Unnamed: 0')
df_test = pd.read_csv('/kaggle/input/blackfriday-scaled/BF_test_scaled.csv',index_col='Unnamed: 0')

In [7]:
df_train

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,1,0.294864,1,2,0,-0.610809,-0.293605,0.368792,8370.0
1,0,1,0.294864,1,2,0,-1.118912,-0.759362,-0.361176,15200.0
2,0,1,0.294864,1,2,0,1.675656,-0.293605,0.368792,1422.0
3,0,1,0.294864,1,2,0,1.675656,1.103664,0.368792,1057.0
4,1,7,1.214734,3,4,0,0.659449,-0.293605,0.368792,7969.0
...,...,...,...,...,...,...,...,...,...,...
550063,1,6,0.754799,2,1,1,3.708069,-0.293605,0.368792,368.0
550064,0,3,-1.084943,3,3,0,3.708069,-0.293605,0.368792,371.0
550065,0,3,1.061423,2,4,1,3.708069,-0.293605,0.368792,137.0
550066,0,7,-1.084943,3,2,0,3.708069,-0.293605,0.368792,365.0


In [8]:
df_test.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
550068,1,5,-0.165072,2,2,1,-1.118912,0.405029,0.368792,NaN
550069,1,3,1.368046,3,0,0,-0.610809,-0.992240,0.368792,NaN
550070,0,4,-1.084943,2,4,1,-0.102705,1.103664,0.368792,NaN
550071,0,4,-1.084943,2,4,1,-0.356757,-0.060727,0.368792,NaN
550072,0,3,-1.084943,3,1,0,-0.356757,-0.992240,-1.091145,NaN


In [9]:
X = df_train.drop('Purchase',axis=1)
y = df_train['Purchase']

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

### **Training with only basic model objects**


In [11]:
models = {
        'RandomForestRegressor': RandomForestRegressor(),
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'Lasso': Lasso(),
        'XGBRegressor': XGBRegressor(),
        'CatBoostRegressor' : CatBoostRegressor(verbose=False),
        'Gradient_Boosting' : GradientBoostingRegressor(),
        'Adaboost_Regressor' : AdaBoostRegressor(),
        
    }


In [14]:
def evaluate_default_models(X_train, y_train, X_test, y_test,models):
    
    
    report_nogscv = []

    for model_name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        end_time = time.time()
        
        
        training_time = end_time - start_time
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_model_score = r2_score(y_train, y_train_pred)
        test_model_score = r2_score(y_test, y_test_pred)
        Mae = mean_absolute_error(y_test, y_test_pred)
        RMse = mean_squared_error(y_test, y_test_pred,squared= False)

        report_nogscv.append({
            'Model_name': model_name,
            'R2_Score_test': test_model_score,
            'R2_Score_train': train_model_score,
            'MAE': Mae,
            'RMSE': RMse,
            'Training_Time': training_time/60
            
        })

    return report_nogscv

In [17]:
report_nogs = evaluate_default_models(X_train, y_train, X_test, y_test,models)

In [18]:
report_nogs

[{'Model_name': 'RandomForestRegressor',
  'R2_Score_test': 0.6295437578584442,
  'R2_Score_train': 0.780788539658765,
  'MAE': 2220.8854164340164,
  'RMSE': 3050.9261219781047,
  'Training_Time': 1.2372452100118},
 {'Model_name': 'LinearRegression',
  'R2_Score_test': 0.13009752050368628,
  'R2_Score_train': 0.13116158704004344,
  'MAE': 3576.6193901056677,
  'RMSE': 4675.181946658147,
  'Training_Time': 0.0025675058364868163},
 {'Model_name': 'Ridge',
  'R2_Score_test': 0.13009752510732786,
  'R2_Score_train': 0.13116158703917002,
  'MAE': 3576.6198588074476,
  'RMSE': 4675.1819342873005,
  'Training_Time': 0.0010080973307291666},
 {'Model_name': 'Lasso',
  'R2_Score_test': 0.1300963059940763,
  'R2_Score_train': 0.13116097207556932,
  'MAE': 3576.6801082617985,
  'RMSE': 4675.185210271918,
  'Training_Time': 0.002364333470662435},
 {'Model_name': 'XGBRegressor',
  'R2_Score_test': 0.6660315347704567,
  'R2_Score_train': 0.6795480631107598,
  'MAE': 2163.14527613874,
  'RMSE': 2896.7

In [19]:
report_nogs_df = pd.DataFrame(report_nogs).sort_values(by='RMSE',ascending=True)

In [20]:
report_nogs_df

,Model_name,R2_Score_test,R2_Score_train,MAE,RMSE,Training_Time
4,XGBRegressor,0.666032,0.679548,2163.145276,2896.783001,0.460477
5,CatBoostRegressor,0.665757,0.677714,2165.890656,2897.973900,0.973721
6,Gradient_Boosting,0.642872,0.649617,2267.903471,2995.540704,0.376484
0,RandomForestRegressor,0.629544,0.780789,2220.885416,3050.926122,1.237245
7,Adaboost_Regressor,0.403405,0.408696,3050.921229,3871.710911,0.251031
2,Ridge,0.130098,0.131162,3576.619859,4675.181934,0.001008
1,LinearRegression,0.130098,0.131162,3576.619390,4675.181947,0.002568
3,Lasso,0.130096,0.131161,3576.680108,4675.185210,0.002364


In [21]:
report_nogs_df.to_csv('/kaggle/working/Default_Model_Report.csv')

### **XGB, catboost  performed better, we will try using hyperparameter tuning to see if we can improve the score**

## **Training with hyperparameter tuning**

In [51]:
models_params= {


                "XGBRegressor":
                            {
                            'model_obj' : XGBRegressor(tree_method = 'gpu_hist'),
                            'params' :              {
                                                        'learning_rate':[0.1,0.15],
                                                        'max_depth': [10,20],
                                                        'gamma': [0,0.1],
                                
                                                    },
                            },


                  "CatBoosting Regressor": 
                            {
                            'model_obj' : CatBoostRegressor(verbose=False),
                            'params' :          {
                                                    'depth': [10,15],
                                                    'learning_rate': [0.1,0.05,],
                                                    'iterations': [1000,1500]
                                                    },
                            },



            }

In [52]:
import time

In [53]:
class ModelEvaluator:
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.best_models = {}

    def evaluate_models_hypeparam(self, models_params):
        report = []

        for model_name, m_p in models_params.items():
            model = m_p['model_obj']
            params = m_p['params']

            gs = GridSearchCV(model, params, cv=4, scoring='neg_mean_squared_error')

            start_time = time.time()
            gs.fit(self.X_train, self.y_train)
            end_time = time.time()

            training_time = end_time - start_time

            best_model = gs.best_estimator_

            y_train_pred = best_model.predict(self.X_train)
            y_test_pred = best_model.predict(self.X_test)

            test_model_score = r2_score(self.y_test, y_test_pred)
            train_model_score = r2_score(self.y_train, y_train_pred)

            RMSE = mean_squared_error(self.y_test, y_test_pred, squared=False)

            report.append({
                'Model_name': model_name,
                'RMSE': RMSE,
                'RMSE_GS': np.sqrt(-gs.best_score_),
                'R2_test': test_model_score,
                'R2_train': train_model_score,
                'Best_Params': gs.best_params_,
                'Training_Time': training_time/60
            })

            self.best_models[model_name] = best_model

        return report

    def get_best_model(self, model_name):
        return self.best_models.get(model_name)

In [ ]:
report_gs = evaluate_models_hypeparam(X_train,y_train,X_test,y_test,models_params)

In [ ]:
pd.DataFrame(report_gs)

,Model_name,RMSE,RMSE_GS,R2_test,R2_train,Best_Params,Training_Time
0,XGBRegressor,2890.122415,2885.185044,0.667566,0.72502,"{'gamma': 0, 'learning_rate': 0.15, 'max_depth...",30.074064
1,CatBoosting Regressor,2895.161841,2888.450660,0.666405,0.71932,"{'depth': 10, 'iterations': 1000, 'learning_ra...",138.354973
